<a href="https://colab.research.google.com/github/anyelyra/TaskWeek7/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Anyelyra Kantata

NPM: 2206048625

# Look up the model

***Look up sentiment analysis models on HuggingFace***

Model: https://huggingface.co/ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa

# Running the model

In [ ]:
# SET UP ENVIRONMENT

!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
# IMPORT LIBRARIES
# LOADING THE MODEL

import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from sklearn.metrics import accuracy_score

In [ ]:
# LOADING THE RoBERTa MODEL

model_name = "ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/822k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/468k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/990 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

## Synthetic Dataset -- *from ChatGPT*

In [ ]:
data = {
    "text": [
        "Saya sangat menyukai produk ini!",      # Positive
        "Ini adalah pengalaman terburuk yang pernah ada.",  # Negative
        "Luar biasa sekali!",                    # Positive
        "Saya sangat kecewa.",                   # Negative
        "Biasa saja, tidak terlalu bagus tapi juga tidak buruk.", # Neutral
        "Pembelian terbaik yang pernah saya lakukan!", # Positive
        "Mengecewakan sekali.",                  # Negative
        "Ini melebihi harapan saya.",            # Positive
        "Produk ini tidak sesuai dengan deskripsi.", # Negative
        "Saya merasa biasa saja.",               # Neutral
        "Kualitasnya lumayan bagus.",            # Neutral
        "Sungguh luar biasa dan mengesankan!",   # Positive
        "Pengiriman sangat lambat.",             # Negative
        "Pelayanan baik, namun produk biasa saja.", # Neutral
        "Barang rusak ketika diterima.",         # Negative
        "Sesuai dengan ekspektasi saya.",        # Positive
        "Produk sangat tidak memuaskan.",        # Negative
        "Layanan pelanggan sangat membantu.",    # Positive
        "Saya tidak akan membeli lagi.",         # Negative
        "Produk ini sangat berguna dan berkualitas tinggi!" # Positive
    ],
    "label": [0, 1, 0, 1, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 1, 0, 1, 0, 1, 0]  # 0: Positive, 1: Negative, 2: Neutral
}

# Convert to HuggingFace Dataset
dataset = Dataset.from_dict(data)

In [ ]:
def preprocess(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

dataset = dataset.map(preprocess, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

## Model

In [ ]:
# MODEL YANG DIGUNAKAN

# Base Mode
base_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Yang telah diimplentasikan Attention Transformer
# Custom Attention Model
class CustomAttentionModel(nn.Module):
    def __init__(self, model_name):
        super(CustomAttentionModel, self).__init__()
        self.base_model = AutoModelForSequenceClassification.from_pretrained(model_name)
        hidden_size = self.base_model.config.hidden_size
        self.attention = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=8)
        self.classifier = nn.Linear(hidden_size, self.base_model.config.num_labels)

    def forward(self, input_ids, attention_mask):
        base_outputs = self.base_model.roberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = base_outputs.last_hidden_state

        # Gunakan CLS token sebagai representasi
        cls_output = hidden_states[:, 0, :]  # Ambil output dari token CLS
        logits = self.classifier(cls_output)  # Langsung klasifikasi menggunakan CLS
        return logits

custom_model = CustomAttentionModel(model_name)

# Evaluasi dan Hasil Prediksi

In [ ]:
# Fungsi untuk Evaluasi dan Menampilkan Prediksi
def evaluate_model(model, dataset):
    model.eval()
    preds = []
    results = []

    with torch.no_grad():
        for i in range(len(dataset)):
            input_ids = dataset[i]["input_ids"].unsqueeze(0)
            attention_mask = dataset[i]["attention_mask"].unsqueeze(0)

            # Dapatkan logits dan prediksi
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits if hasattr(outputs, 'logits') else outputs

            pred_label = torch.argmax(logits, dim=1).item()
            true_label = dataset["label"][i]

            # Simpan hasil untuk analisis
            preds.append(pred_label)
            results.append({
                "text": dataset["text"][i],
                "predicted_label": pred_label,
                "true_label": true_label
            })

    # Hitung akurasi
    accuracy = accuracy_score(dataset["label"], preds)

    return accuracy, results

## *untuk base model*

In [ ]:
# Evaluasi Model Dasar
base_accuracy, base_results = evaluate_model(base_model, dataset)
print("Base Model Accuracy:", base_accuracy)

# Tampilkan Hasil Prediksi Model Dasar
print("\nPredictions from Base Model:")
for result in base_results:
    print(f"Text: {result['text']}")
    print(f"Predicted Label: {result['predicted_label']}, True Label: {result['true_label']}")
    print("---")

Base Model Accuracy: 0.5

Predictions from Base Model:
Text: Saya sangat menyukai produk ini!
Predicted Label: 0, True Label: 0
---
Text: Ini adalah pengalaman terburuk yang pernah ada.
Predicted Label: 2, True Label: 1
---
Text: Luar biasa sekali!
Predicted Label: 0, True Label: 0
---
Text: Saya sangat kecewa.
Predicted Label: 2, True Label: 1
---
Text: Biasa saja, tidak terlalu bagus tapi juga tidak buruk.
Predicted Label: 0, True Label: 2
---
Text: Pembelian terbaik yang pernah saya lakukan!
Predicted Label: 0, True Label: 0
---
Text: Mengecewakan sekali.
Predicted Label: 2, True Label: 1
---
Text: Ini melebihi harapan saya.
Predicted Label: 0, True Label: 0
---
Text: Produk ini tidak sesuai dengan deskripsi.
Predicted Label: 2, True Label: 1
---
Text: Saya merasa biasa saja.
Predicted Label: 2, True Label: 2
---
Text: Kualitasnya lumayan bagus.
Predicted Label: 0, True Label: 2
---
Text: Sungguh luar biasa dan mengesankan!
Predicted Label: 0, True Label: 0
---
Text: Pengiriman sang

## *untuk model custom*

In [ ]:
# Evaluasi Model dengan Attention
custom_accuracy, custom_results = evaluate_model(custom_model, dataset)
print("\nCustom Model with Attention Accuracy:", custom_accuracy)

# Tampilkan Hasil Prediksi Model dengan Attention
print("\nPredictions from Custom Attention Model:")
for result in custom_results:
    print(f"Text: {result['text']}")
    print(f"Predicted Label: {result['predicted_label']}, True Label: {result['true_label']}")
    print("---")


Custom Model with Attention Accuracy: 0.5

Predictions from Custom Attention Model:
Text: Saya sangat menyukai produk ini!
Predicted Label: 2, True Label: 0
---
Text: Ini adalah pengalaman terburuk yang pernah ada.
Predicted Label: 1, True Label: 1
---
Text: Luar biasa sekali!
Predicted Label: 1, True Label: 0
---
Text: Saya sangat kecewa.
Predicted Label: 1, True Label: 1
---
Text: Biasa saja, tidak terlalu bagus tapi juga tidak buruk.
Predicted Label: 2, True Label: 2
---
Text: Pembelian terbaik yang pernah saya lakukan!
Predicted Label: 2, True Label: 0
---
Text: Mengecewakan sekali.
Predicted Label: 1, True Label: 1
---
Text: Ini melebihi harapan saya.
Predicted Label: 1, True Label: 0
---
Text: Produk ini tidak sesuai dengan deskripsi.
Predicted Label: 1, True Label: 1
---
Text: Saya merasa biasa saja.
Predicted Label: 1, True Label: 2
---
Text: Kualitasnya lumayan bagus.
Predicted Label: 2, True Label: 2
---
Text: Sungguh luar biasa dan mengesankan!
Predicted Label: 2, True Labe

# Kesimpulan


Model dasar dan model custom dengan attention menunjukkan akurasi yang sama, yaitu 0.5.

Hal ini menunjukkan bahwa keduanya mungkin tidak mempelajari pola dengan baik. Kemungkinan penyebabnya adalah ketidakseimbangan data atau implementasi Attention yang kurang optimal.

Untuk meningkatkan akurasi, dapat dilakukan beberapa cara, seperti memeriksa distribusi label, hyperparameter-tuning, dan analisis output logits.

Selain itu, gunakan metrik tambahan seperti F1-score untuk evaluasi yang lebih komprehensif.